In [ ]:
%pip install openai
%pip install scikit-learn

In [ ]:
from IPython.display import Image, display
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from openai import OpenAI
import os
from PIL import Image

client = OpenAI(
    api_key = os.environ["OPENAI_API_KEY"],
)

In [ ]:
example_image_filepath = "datasets/images/COCO_val2014_000000000400.jpg"
example_image = Image.open(example_image_filepath)
display(example_image)

In [ ]:
import base64
from mimetypes import guess_type

def local_image_to_data_url(image_path):
    mime_type, _ = guess_type(image_path)
    if mime_type is None:
        mime_type = 'application/octet-stream'

    with open(image_path, "rb") as image_file:
        base64_encoded_data = base64.b64encode(image_file.read()).decode('utf-8')

    return f"data:{mime_type};base64,{base64_encoded_data}"

data_url = local_image_to_data_url(example_image_filepath)
print("Data URL:", data_url)

In [ ]:
describe_system_prompt = '''
    You are a system generating descriptions for furniture items, decorative items, or furnishings on an e-commerce website.
    Provided with an image and a title, you will describe the main item that you see in the image, giving details but staying concise.
    You can describe unambiguously what the item is and its material, color, and style if clearly identifiable.
    If there are multiple items depicted, refer to the title to understand which item you should describe.
    '''

def describe_image(img_url):
    response = client.chat.completions.create(
    model="gpt-4-vision-preview",
    temperature=0.2,
    messages=[
        {
            "role": "system",
            "content": describe_system_prompt
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": img_url,
                },
            ],
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "What's in this image?"
                }
            ]
        }
    ],
    max_tokens=300,
    )

    return response.choices[0].message.content

In [ ]:
print(describe_image(data_url))